In [1]:
import sys
from gapfill_utilities import utils as gf_utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
sys.path.append('/home/blattms1/projects/gapfill/4CL_JAK2/likelihood_model/src/')
import utils as lm_utils

In [2]:
def get_gapfill_likelihoods(gapfill,error_rate_dicts,alleles):
    likelihoods = lm_utils.get_likelihoods_of_true_allele(gapfill, error_rate_dicts)
    likelihood_list = {}
    for allele in alleles:
        if allele in likelihoods:
            likelihood_list[allele] = likelihoods[allele]
        else:
            likelihood_list[allele] = 10e-10
    return likelihood_list
error_rate_dir = '/home/blattms1/projects/gapfill/4CL_JAK2/likelihood_model/error_rate_dicts_HB'
error_rate_dicts = lm_utils.get_error_rate_dicts(error_rate_dir)

In [3]:
feature_set = 'feature_set_all.csv'


In [ ]:
for lib in ['1plex','1','2','4plex']:
    if lib == '4plex':
        BCs = ['BC001', 'BC002', 'BC003', 'BC004']
    elif lib == '1plex':
        BCs = ['BC001']
    else:
        BCs = ['BC001', 'BC002', 'BC003', 'BC004', 'BC005', 'BC006', 'BC007', 'BC008', 'BC009', 'BC010', 'BC011', 'BC012', 'BC013', 'BC014', 'BC015', 'BC016']

    for BC in BCs:
        gf_dirs = {}

        ### first get probe_reads to use for the patient
        if lib == '4plex':
            adata_path = '/data1/lareauc/users/blattms/gapfill/wta_adata/20250318_rerun_JAK2_patients_' + BC + '.h5ad'
            gf_dirs[0] = '/data1/lareauc/projects/gapfill/analysis/20250605_MPN_16plex/gf_MPN_4plex_1/BC' + str(int(BC.replace('BC',''))) + '_giftwrap/'
            gf_dirs[1] = '/data1/lareauc/projects/gapfill/analysis/20250605_MPN_16plex/gf_MPN_4plex_2/BC' + str(int(BC.replace('BC',''))) + '_giftwrap/'
        elif lib == '1plex':
            adata_path = '/data1/lareauc/users/blattms/gapfill/wta_adata/20241114_rerun_JAK2_patients_' + BC + '.h5ad'
            gf_dirs[0] = '/data1/lareauc/projects/gapfill/analysis/20250605_MPN_16plex/gf_MPN_20241114/BC' + str(int(BC.replace('BC',''))) + '_giftwrap/'
        else:
            adata_path = '/data1/lareauc/users/blattms/gapfill/wta_adata/20250605_MPN_' + lib + '_' + BC + '.h5ad'
            gf_dirs[0] = '/data1/lareauc/projects/gapfill/analysis/20250605_MPN_16plex/gf_MPN_16plex_part' + lib + '_1/BC' + str(int(BC.replace('BC',''))) + '_giftwrap/'
            gf_dirs[1] = '/data1/lareauc/projects/gapfill/analysis/20250605_MPN_16plex/gf_MPN_16plex_part' + lib + '_2/BC' + str(int(BC.replace('BC',''))) + '_giftwrap/'

        min_percent_supporting = 0.9
        collapse_across_probes = True
        for key,gf_dir in gf_dirs.items():
            if key == 0:
                probe_reads = gf_utils.get_input_probe_reads(gf_dir, read_threshold = 0, cell_barcode_suffix = '-' + str(key), adata_path = adata_path, min_percent_supporting=min_percent_supporting, collapse_across_probes=collapse_across_probes)
            else:
                probe_reads = pd.concat([probe_reads, gf_utils.get_input_probe_reads(gf_dir, read_threshold = 0, cell_barcode_suffix = '-' + str(key), adata_path = adata_path, min_percent_supporting=min_percent_supporting, collapse_across_probes=collapse_across_probes)], ignore_index=True)

        sample = BC + '_' + lib

        alt_gapfill_set = pd.read_csv(feature_set)

        sample = BC + '_' + lib
        alt_gapfill_set = alt_gapfill_set.loc[alt_gapfill_set['sample'] == sample]

        if len(alt_gapfill_set) > 0:
            ## reformat alt gapfill set
            manifest = gf_utils.get_manifest(gf_dirs[0])
            probe_name_to_probe_idx = dict(zip(manifest['name'], manifest['index']))
            alt_gapfill_set['probe_idx'] = alt_gapfill_set['original_name'].map(probe_name_to_probe_idx)
            
            alt_gapfill_set = pd.concat([
                alt_gapfill_set[['probe_idx', 'name', 'gapfill']].rename(columns={'gapfill': 'gapfill_value'}).assign(variant=True),
                alt_gapfill_set[['probe_idx', 'name', 'gapfill_from_transcriptome']].rename(columns={'gapfill_from_transcriptome': 'gapfill_value'}).assign(variant=False)
            ], ignore_index=True)

            alt_gapfill_set = alt_gapfill_set.dropna(subset='probe_idx')
            # print(alt_gapfill_set)
            
            ### from now on, we only need to consider probes that have an alt gapfill
            probe_reads = probe_reads.loc[probe_reads['probe_idx'].isin(alt_gapfill_set['probe_idx'])]

            unique_gapfill_list = probe_reads[['probe_idx','gapfill']].drop_duplicates()

            possible_alleles = {}
            for probe_idx in probe_reads['probe_idx'].unique():
                possible_alleles[probe_idx] = alt_gapfill_set.loc[alt_gapfill_set['probe_idx'] == probe_idx, 'gapfill_value'].unique()

            unique_gapfill_list['likelihood'] = unique_gapfill_list.apply(
                lambda row: get_gapfill_likelihoods(row['gapfill'], error_rate_dicts, possible_alleles[row['probe_idx']]), axis=1
            )

            for probe_idx in possible_alleles.keys():
                likelihood_keys = possible_alleles[probe_idx]
                for key in likelihood_keys:
                    unique_gapfill_list.loc[unique_gapfill_list['probe_idx'] == probe_idx,key] = unique_gapfill_list.loc[unique_gapfill_list['probe_idx'] == probe_idx, 'likelihood'].apply(lambda x: x.get(key, None))
                
            unique_gapfill_list = unique_gapfill_list.drop(columns=['likelihood'])

            unique_gapfill_list.to_csv('likelihood_tables_' + lib + '/' + BC + '_gapfill_likelihoods.csv', index=False)

1756198 UMIs found
Collapsing UMIs across probes, 1756198 UMIs remaining (100.00%)
Filtering probe reads based on read threshold (0) and min percent supporting (0.9), 1749625 UMIs remaining (99.63%)
Filtering cells based on min counts (0) and genes (0) in WTA
Filtering probe reads based on cell barcodes in adata, 1642071 UMIs remaining (93.50%)
2010873 UMIs found
Collapsing UMIs across probes, 2010873 UMIs remaining (100.00%)
Filtering probe reads based on read threshold (0) and min percent supporting (0.9), 2003477 UMIs remaining (99.63%)
Filtering cells based on min counts (0) and genes (0) in WTA
Filtering probe reads based on cell barcodes in adata, 1875420 UMIs remaining (93.26%)
